# CIFAR-10 distributed training using Horovod

<font color='red'> <h3>To run the code in this notebook, go to [launch_horovod.ipynb](launch_horovod.ipynb). Running this notebook will not enable Horovod and results in errors!</h3></font>

After reading up [basics about Hops Notebook and local training models on Hops](../TensorFlow/cifar10_on_hops.ipynb) it's time to scale out our training. We already discussed [multi-gpu training with TensorFlow](../TensorFlow/multigpu/Multi-gpu_training_cifar.ipynb) but the key difference here is using [Horovod](https://github.com/uber/horovod) library to parallelize the training. The reason to do this is great scaling capabilities this library has in compare to traditional methods due to using MPI and Ring allreduce concepts for parallelization.

> If you want a simpler dataset such as **mnist**, we have a [notebook](mnist.ipynb) for that too

## Importing Libraries

In [ ]:
import tensorflow as tf

from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
import horovod.tensorflow as hvd

from hops import tensorboard
from hops import hdfs
import os

## Initilizaing hyper-parameters

Beside usual hyper-parameters for deep neural network like number of epochs and batch size, for defining the path to our data and log directory for tensorboard, note the usage of two handy modules in `hops` library:
1. `tensorboard.logdir()`: Get the path to your log directory so you can save checkpoints and summaries in this folder.
2. `hdfs.project_path()`: Get the path to your main Dataset folder, after that you need to specify project name and any folder inside.

## Input Data function

For processing the data using `tf.Dataset` api, we provide a utility function to build the `iterator` over the train or test dataset depending on which file names we provide as parameter. 

In [ ]:
tf.logging.set_verbosity(tf.logging.DEBUG)
num_input = 32*32*3
batch_size = 2048
num_classes = 10

# Adjust number of epochs based on number of GPUs.
epochs = 10

last_step = 100000
evaluation_step = 10

lr = 0.003

logdir = tensorboard.logdir()
data_dir = hdfs.project_path() + "TestJob/data/cifar10/"
train_filenames = [data_dir + "train/train.tfrecords"]
validation_filenames = [data_dir + "validation/validation.tfrecords"]
test_filenames = [data_dir + "test/eval.tfrecords"]

## Building the model

In [ ]:
# Convolutional network building
def model(features):
    net = tf.reshape(features, [-1, 32, 32, 3])
    tf.summary.image("image", net)
    network = conv_2d(net, 32, 3, activation='relu')
    network = max_pool_2d(network, 2)
    network = conv_2d(network, 64, 3, activation='relu')
    network = conv_2d(network, 64, 3, activation='relu')
    network = max_pool_2d(network, 2)
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.5)
    logits  = fully_connected(network, 10, activation='linear')

    return logits

## Integration with Horovod

Until now everything was quite standard in Deep learning techniques and Hops concepts. For running our code using Horovod however, we need to modify just few lines of our code. This few lines will enable us to scale our model potentially on hundreds of GPUs very well, providing optimal data input pipeline and fast GPUs connection between each other. 



### 7 Step to scale out with Horovod:

#### 1. Initilization using `hvd.init()`
#### 2. Making our optimizer Horovod-friendly:
```python
opt = tf.train.RMSPropOptimizer(lr)

# Wrap with Distributed Optimizer
opt = hvd.DistributedOptimizer(opt)
```

####  3. Broadcasting: 
Broadcast initial variable states from rank 0 to all other processes. This is necessary to ensure consistent initialization of all workers when training is started with random weights or restored from a checkpoint.

```python

```
####  4. Providing `Config`: 
Pin GPU to be used to process local rank (one GPU per process)

```python
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = str(hvd.local_rank())
```
####  5. Mind the rank:
As now we have potential tens or hundreds of GPUs running the same code, we need to take care of checkpointing and writing summaries. More specifically, we need to do this actions on only one machine. For this we need to check the rank of GPU before each of these actions:

```python
# Creating different directories for writing train and test summaries
if hvd.local_rank()==0:
    if not os.path.exists(logdir + '/train'):
        os.mkdir(logdir + '/train')
    if not os.path.exists(logdir + '/test'):
        os.mkdir(logdir + '/test')         
```

And for actually writing them during training:
```python

if hvd.local_rank()==0:
    train_writer = tf.summary.FileWriter(logdir + '/train', sess.graph)
    test_writer = tf.summary.FileWriter(logdir+ '/test', sess.graph)
    
# While in the loop
if hvd.local_rank()==0:
    train_writer.add_summary(summary, step)
```
#### 6. Run broadcasting op:
Don't forget to run broadcasting operation which we defined earlier after initialization of variables:

```python
init.run()
bcast.run()
```
#### 7. Launch it:

Congrats, we finished adapting our code to use Horovod, but for actually start training, we need to have another notebook. Refer to [launch_horovod.ipynb](launch_horovod.ipynb) notebook for further instruction on how to run and monitor your training process. 

<font color='red'> <h4>To launch this notebook, go to [launch_horovod.ipynb](launch_horovod.ipynb). Running this notebook will not enable Horovod and results in errors!</h4></font>


In [ ]:
def main(_):
    # Initialize Horovod
    hvd.init()

    # Build model...
    with tf.name_scope('input'):
        # Placeholders for data and labels
        X = tf.placeholder(shape=(None, 32, 32, 3), dtype=tf.float32)
        Y = tf.placeholder(shape=(None, 10), dtype=tf.float32)

    preds = model(X)
    # Defining other ops using Tensorflow
    with tf.name_scope('Loss'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=Y))
        tf.summary.scalar('loss', loss)

    opt = tf.train.RMSPropOptimizer(lr)

    # Wrap with Distributed Optimizer
    opt = hvd.DistributedOptimizer(opt)

    global_step = tf.contrib.framework.get_or_create_global_step()
    train_op = opt.minimize(loss, global_step=global_step)

    with tf.name_scope('Accuracy'):
        accuracy = tf.reduce_mean(
            tf.cast(tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1)), tf.float32),
            name='acc')
        # create a summary for our accuracy
        tf.summary.scalar("accuracy", accuracy)


    # Merge all the summaries and write them out to log directory
    merged = tf.summary.merge_all()

    # Initializing the variables
    init = tf.global_variables_initializer()

    # Broadcast initial variable states from rank 0 to all other processes.
    # This is necessary to ensure consistent initialization of all workers when
    # training is started with random weights or restored from a checkpoint.
    bcast = hvd.broadcast_global_variables(0)

    # Get train and test iterators
    train_input_iterator = data_input_fn(train_filenames, batch_size=batch_size)
    test_input_iterator =  data_input_fn(test_filenames)

    # Pin GPU to be used to process local rank (one GPU per process)
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.visible_device_list = str(hvd.local_rank())

    # Only rank 0 needs to create the directories
    if hvd.local_rank()==0:
        if not os.path.exists(logdir + '/train'):
            os.mkdir(logdir + '/train')
        if not os.path.exists(logdir + '/test'):
            os.mkdir(logdir + '/test')

    with tf.Session(config=config) as sess:
        print('Initializing...')
        init.run()
        bcast.run()
        
        if hvd.local_rank()==0:
            train_writer = tf.summary.FileWriter(logdir + '/train', sess.graph)
            test_writer = tf.summary.FileWriter(logdir+ '/test', sess.graph)
        
        step = 0
        while step < last_step:
            # Run a training step synchronously.
            images_, labels_ = sess.run(train_input_iterator.get_next())
            _, loss_, step, summary = sess.run([train_op, loss, global_step, merged],
                feed_dict={X: images_, Y: labels_})
            
            if hvd.local_rank()==0:
                train_writer.add_summary(summary, step)
            print('Step: {}, Loss:{}'.format(str(step), str(loss_)))
            if step % evaluation_step == 0:
                images_test, labels_test = sess.run(test_input_iterator.get_next())
                acc_, loss_, step, summary = sess.run([accuracy, loss, global_step, merged],
                    feed_dict={X: images_test, Y: labels_test})
                if hvd.local_rank()==0:
                    test_writer.add_summary(summary, step)
                print("Step:", '%03d' % step,
                      "Loss:", str(loss_),
                      "Accuracy:", str(acc_))
                

## Don't forget:
To call the main function if you wrapped your model logic in the `main()`

In [ ]:
if __name__ == "__main__":
    print("Starting!")
    tf.app.run()